In [1]:
import pandas as pd

# NASBench201

In [2]:
from naslib.search_spaces.core import Metric
from tqdm import tqdm
from naslib.utils import get_dataset_api
from naslib.search_spaces import nasbench201 as nb201
from naslib.search_spaces.nasbench201.graph import NasBench201SearchSpace

from nas_201_api import NASBench201API

cifar10_dataset_api = get_dataset_api("nasbench201", 'cifar10')
cifar100_dataset_api = get_dataset_api("nasbench201", 'cifar100')
real_nb201_api = NASBench201API("../../naslib/data/NAS-Bench-201-v1_0-e61699.pth", verbose=False)
from xautodl.models import get_cell_based_tiny_net
from naslib.search_spaces.nasbench201.conversions import convert_str_to_op_indices, convert_op_indices_to_naslib


0it [00:00, ?it/s]


KeyError: 'cifar100'

In [13]:
import numpy as np

df = pd.DataFrame(columns=["arch_id", "arch_str", "params", "cifar_10_val_accuracy"])

for i, arch_str in tqdm(enumerate(real_nb201_api)):
    indices = convert_str_to_op_indices(arch_str)
    objec = NasBench201SearchSpace()
    convert_op_indices_to_naslib(indices, objec)
    cifar_10_val_accuracy = objec.query(dataset='cifar10', metric=Metric.VAL_ACCURACY, dataset_api=cifar10_dataset_api)
    cifar_100_val_accuracy = objec.query(dataset='cifar100', metric=Metric.VAL_ACCURACY, dataset_api=cifar100_dataset_api)

    id = real_nb201_api.query_index_by_arch(arch_str)
    config = real_nb201_api.get_net_config(id, 'cifar10-valid')  # obtain the network configuration for the n-th architecture on the CIFAR-10 dataset
    network = get_cell_based_tiny_net(config)  # create the network from configuration
    params = sum([np.prod(p.size()) for p in network.parameters()])

    df = df.append({"arch_id": id, "arch_str": arch_str, "params": params, "cifar_10_val_accuracy": cifar_10_val_accuracy}, ignore_index=True)

15625it [52:34,  4.95it/s] 


In [14]:
df.to_csv("nasbench201.csv")

In [15]:
df.head()

,arch_id,arch_str,params,cifar_10_val_accuracy
0,0,|avg_pool_3x3~0|+|nor_conv_1x1~0|skip_connect~...,129306,81.90
1,1,|nor_conv_3x3~0|+|nor_conv_3x3~0|avg_pool_3x3~...,802426,90.39
2,2,|avg_pool_3x3~0|+|nor_conv_3x3~0|nor_conv_3x3~...,559386,81.93
3,3,|avg_pool_3x3~0|+|skip_connect~0|none~1|+|none...,73306,83.78
4,4,|skip_connect~0|+|skip_connect~0|nor_conv_1x1~...,129306,84.26
